In [1]:
from prep_data import Data_prepare
import numpy as np 
import os 
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# import prep_data

###  Data preprocessing 
1. covert mat data to numpy array 
2. combine male and female data to one array 
3. if needed, remove certain rows 
4. get postive and negative of Xs 
    - order difference or net vs neg 
5. reshape each X: 128*8*2 = 2048 
    - input: data 
6. generate labels for x 
    - input: data and label_type (0:zeros, 1: ones )
7. combine net and neg 
    - input: netX,netY,negX,negY

In [2]:
def netural_vs_negative_helper(data,position):
    arr = []
    index = 0
    for i in range(data.shape[0]):
#         print(data[i].shape)
        temp = []
        for j in position:
            for k in data[i][j[0]][j[1]]:
                temp.append(k)
        arr.append(np.array(temp))
    return np.array(arr)

In [3]:
def get_x_y(stress):
    dp = Data_prepare() 
    pos = [[0,3],[0,2],[0,1],[0,0]]
    neg = [[1,3],[1,2],[1,1],[1,0]]
    # netural vs postive 
    # pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
    # neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]] 
    m_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat" 
    f_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
    # index = [11,18,36]
    df = dp.combine_male_female(m_file,f_file)
    # df = dp.remove_person(df,index)
    pos_df = dp.get_pos_or_neg(df,pos)
    neg_df = dp.get_pos_or_neg(df,neg)
    pos_sequeezed = dp.squeeze_feature_size(pos_df)
    neg_sequeezed = dp.squeeze_feature_size(neg_df)
    pos_labels = dp.generate_labels(pos_sequeezed,1)
    neg_labels = dp.generate_labels(neg_sequeezed,0)
    x,y = dp.combine_net_neg(pos_sequeezed,pos_labels,neg_sequeezed,neg_labels)
    return dp.choose_x_y_stress(stress,x,y)
    

In [5]:
# model = RandomForestClassifier(n_estimators=150) 
model = LogisticRegression(solver='saga', 
                                penalty='l1',
                                C=100,
                                max_iter=1000)

In [6]:
def get_x_train_test(data,index):
    x_test = data[index]
    x_train = np.delete(data,index,axis=0)
    return x_train,x_test.reshape(1,-1)

# get y train and test set for witin CV 
def get_y_train_test(data,index):
    y_test = data[index]
    y_train = np.delete(data,index,axis=0)
    return y_train.astype(int).ravel(),y_test.astype(int)
def model_train(x_train,x_test,y_train,y_test,model):
    model.fit(x_train,y_train.ravel())
    y_pred = model.predict(x_test)
    return accuracy_score(y_test,y_pred),model.coef_

def within_train(x,y,model):
    # with_in
    all_accuracy = []
    avg_accuracy = []
    all_coefs = []
    # train a model 
    # return y_test, y_pred 
    for d in range(x.shape[0]):
        # training set and test set
        accuracy =[]
        print("within # person:",d)
        temp_coefs =[]
        for i in range(x[d].shape[0]):
            x_train,x_test = get_x_train_test(x[d],i)
            y_train,y_test = get_y_train_test(y[d],i)
            acc,coefs= model_train(x_train,x_test,y_train,y_test,model)
            accuracy.append(acc)
            temp_coefs.append(coefs)
        temp_acc = sum(accuracy)/len(accuracy)
        avg_accuracy.append(temp_acc)
        all_coefs.append(temp_coefs)
        print(temp_acc)
    return avg_accuracy, all_coefs


In [7]:
x,y = get_x_y("m")
x.shape


(12, 36, 2048)

In [8]:
accs,all_coefs = within_train(x,y,model)

within # person: 0


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

0.6666666666666666
within # person: 1


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

0.8055555555555556
within # person: 2


...

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

0.7222222222222222
within # person: 11


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

0.6944444444444444


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [9]:
import json

In [10]:
def write_to_json(avg_acc,coefs,file_name):
    data = {} 
    data['acc'] = avg_acc
    data['coefs'] = coefs
    with open(file_name, 'w') as outfile:
        json.dump(data, outfile)

In [11]:
new_arr = []
for i in range(len(all_coefs)):
    temp_arr = []
    for j in range(len(all_coefs[i])):
        for k in range(len(all_coefs[i][j])):
            temp_arr.append(np.array(all_coefs[i][j][k]))
    new_arr.append(np.array(temp_arr))

In [12]:
new_arr = np.array(new_arr)
new_arr[0].shape
avg_arr =[]
new_arr = np.abs(new_arr)
for i in range(new_arr.shape[0]):
    temp = np.average(new_arr[i],axis=0)
    temp = temp.reshape(128,8,2)
    temp = np.delete(temp,0,axis=2)
    temp = temp.reshape(128,8)
    avg_arr.append(temp)

In [13]:
avg_arr = np.array(avg_arr)
avg_arr.shape

(12, 128, 8)

In [19]:
folder = './results/write_graph/medium/'
write_to_json([],avg_arr.tolist(),folder+"m.json")

In [20]:
from prep_data import Data_prepare
import numpy as np 
import os 
import mne
import io
from matplotlib import pyplot as plt

In [21]:
biosemi_montage = mne.channels.make_standard_montage('biosemi128')
a = biosemi_montage.get_positions()['ch_pos']
pos = []
names=[]
for i in a.keys():
    pos.append(np.array(a[i][:2]))
    names.append(i)
pos = np.array(pos)
names = np.array(names)

In [22]:
def make_one(coefs,pos,names,path):
    # coefs[:,0]
    fig = mne.viz.plot_topomap(coefs, pos, show=False,show_names=True,names=names, contours=0, image_interp="bilinear",ch_type=type,cmap='Blues')
    plt.colorbar(fig[0])
    plt.savefig(path)
    plt.close()

In [23]:
for i in range(avg_arr.shape[0]):
    for j in range(avg_arr[i].shape[1]):
        filename = 'person'+str(i)+"_"+"j.jpg"
        make_one(avg_arr[i,:,j],pos,names,folder+filename)